In [ ]:
# load init moudle file

load "/work/pannote/panbot_note.rb"
nil

In [ ]:
endpoint = [
    ["chainstack-sg","https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"],
    ["chainstack-us","https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"],
    ["chainstack-fullnode","https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"],
]

repeat_time_read = 30


30

In [ ]:
def read_test(contract)
    time = Time.now()
    contract.call.rounds(36545)
    time = Time.now()-time
    return time
end


:read_test

从DeepNote美国服务器，访问ChainStack节点读数据的网络延迟情况

- ["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"] avg time 0.64047529 latency
- ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"] avg time 0.02369665 latency
- ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"] avg time 0.21744277 latency

In [ ]:
endpoint.each do |url|
    # init client and contract
    client = Ethereum::HttpClient.new(url[1])

    contract = Ethereum::Contract.create(
        client: client, 
        name:"PancakePredictionV2", 
        address: "0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA", 
        abi: JSON.parse(open("/work/pannote/PancakePredictionV2.json").read)
    )

    total_time = 0
    total_count = 0

    repeat_time_read.times do |i| 
        time = read_test(contract)
        total_count=total_count+1
        total_time=total_time+time
    end

    puts "#{url} avg time #{ (total_time.to_f / total_count).round(8)} latency"
end

["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"] avg time 0.64047529 latency
["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"] avg time 0.02369665 latency
["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"] avg time 0.21744277 latency


[["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"], ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"], ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"]]

从DeepNote美国服务器，访问ChainStack节点读数据的区块延迟（aka，区块生成之后，要隔多少秒才能取到这个区块）

- ["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"] avg time 0.8217141822s
- ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"] avg time 0.3589499855s
- ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"] avg time 18.200209495s

可以看到，全节点的区块同步不是实时的。

在同样的时间来取，fullnode要比shared node慢1-50个block(甚至更多)

- ["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"] block_number: 15236560  block_time: 2022-02-14 02:14:42.0.0 
- ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"] block_number: 15236560  block_time: 2022-02-14 02:14:42.0.0 
- ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"] block_number: 15236558  block_time: 2022-02-14 02:14:36.0.0 


In [ ]:
# 测试服务器读延迟
endpoint.each do |url|
    # init client and contract
    client = Ethereum::HttpClient.new(url[1])

    puts "#{url}"
    time = Time.now
    save_block_number = 0
    count = 0
    latenct = 0

    while Time.now-time<30 do

    last_block = client.eth_get_block_by_number("latest",false)
    block_number = last_block["result"]["number"].to_i(16)
    block_time = Time.at(last_block["result"]["timestamp"].to_i(16))
    now = Time.now
    
    if block_number !=save_block_number then
        if save_block_number!=0 then
        puts "block_number: #{block_number} latenct: #{now-block_time}  block_time: #{block_time.to_formatted_s(:db)}.#{block_time.subsec.to_f} "
        count = count+1
        latenct = latenct+(now-block_time)
        end
        save_block_number = block_number
    end
    
    sleep(0.01)
    end
        
    puts "avg latenct #{latenct/count} s"
end

["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"]
block_number: 15236476 latenct: 0.499896804  block_time: 2022-02-14 02:10:30.0.0 
block_number: 15236477 latenct: 0.75729934  block_time: 2022-02-14 02:10:33.0.0 
block_number: 15236478 latenct: 1.685361493  block_time: 2022-02-14 02:10:36.0.0 
block_number: 15236479 latenct: 0.655959327  block_time: 2022-02-14 02:10:39.0.0 
block_number: 15236480 latenct: 0.909522813  block_time: 2022-02-14 02:10:42.0.0 
block_number: 15236481 latenct: 0.508273376  block_time: 2022-02-14 02:10:45.0.0 
block_number: 15236482 latenct: 0.743280833  block_time: 2022-02-14 02:10:48.0.0 
block_number: 15236483 latenct: 0.978751563  block_time: 2022-02-14 02:10:51.0.0 
block_number: 15236484 latenct: 0.620362281  block_time: 2022-02-14 02:10:54.0.0 
block_number: 15236485 latenct: 0.858433992  block_time: 2022-02-14 02:10:57.0.0 
avg latenct 0.8217141822 s
["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0

[["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"], ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"], ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"]]

In [ ]:
# 测试服务器读延迟
endpoint.each do |url|
    # init client and contract
    client = Ethereum::HttpClient.new(url[1])

    last_block = client.eth_get_block_by_number("latest",false)
    block_number = last_block["result"]["number"].to_i(16)
    block_time = Time.at(last_block["result"]["timestamp"].to_i(16))
    puts "#{url} block_number: #{block_number}  block_time: #{block_time.to_formatted_s(:db)}.#{block_time.subsec.to_f} "
end

["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"] block_number: 15236613  block_time: 2022-02-14 02:17:21.0.0 
["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"] block_number: 15236613  block_time: 2022-02-14 02:17:21.0.0 
["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"] block_number: 15236558  block_time: 2022-02-14 02:14:36.0.0 


[["chainstack-sg", "https://nd-655-401-702.p2pify.com/31f68016997c5a3c4808f9be6bdaeaeb"], ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"], ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"]]

# 测试服务器写延迟

#### align_time 对齐时间 2.5s

- ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"] 2个区块
- ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"] 2.6个区块

#### align_time 对齐时间 0.5s

- ["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"] 1.8个区块
- ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"] 1.9个区块



In [ ]:
def write_test(client,read_client,contract,align_time)
    last_block = nil
    last_block_time = nil
    last_block_number = nil
    
    save_last_block_number = nil

    loop do
        last_block = read_client.eth_get_block_by_number('latest', false) # get latest block details
        last_block_time = Time.at(last_block["result"]["timestamp"].to_i(16))
        last_block_number = last_block["result"]["number"].to_i(16)
        if not(save_last_block_number==last_block_number) then
            save_last_block_number = last_block_number
            $count = $count+1
            puts "pass time since last block mined #{Time.now()-last_block_time} save_last_block_numbe #{save_last_block_number}  last_block_number #{last_block_number}  compare #{save_last_block_number==last_block_number}"
        end
        next if Time.now()-last_block_time>=align_time

        # align to shift of block interval(s)
        loop do
            break if Time.now()-last_block_time>=align_time
        end
        break
    end
    time = Time.now()
    puts "[align time shift] last_block_number #{last_block_number} - time #{Time.now()-last_block_time} "        


    tx = contract.transact.claim([36545])
    loop do
        tx_eth = read_client.eth_get_transaction_by_hash(tx.id)
        break if !tx_eth.nil? && !tx_eth["result"].nil? && tx_eth["result"]["blockNumber"].present?
        sleep 1
    end    
    tx_hash = tx.address
    tx_detail = read_client.eth_get_transaction_by_hash(tx_hash)
    tx_block_number = tx_detail["result"]["blockNumber"].to_i(16)
    tx_block_time = Time.at(read_client.eth_get_block_by_number(tx_block_number,false)["result"]["timestamp"].to_i(16))



    puts "latency #{tx_block_number-last_block_number} = tx_block_number #{tx_block_number} - last_block_number #{last_block_number}"    
    return tx_block_number-last_block_number
end

:write_test

In [ ]:
# 测试服务器写延迟

endpoint = [
    ["chainstack-us","https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"],
    ["chainstack-fullnode","https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"],
]

repeat_time_write = 10

read_client = Ethereum::HttpClient.new(endpoint.to_h["chainstack-us"])



align_time = 0.5 # send txn 1s after last block

endpoint.each do |url|
    # init client and contract
    client = Ethereum::HttpClient.new(url[1])

    puts "#{url}"

    contract = Ethereum::Contract.create(
        client: client, 
        name:"PancakePredictionV2", 
        address: "0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA", 
        abi: JSON.parse(open("/work/pannote/PancakePredictionV2.json").read)
    )

    bot_prviate_key = "639712feb7c4598b3a8b8f51d53b240bc3e01e8e2d2044063017c26214bb2920"

    gas_premium = 1

    ## config chain_id for EIP-155
    Eth.configure { |c| c.chain_id = client.net_version["result"].to_i }
    
    # update gas prices with gas_premium
    gas_price = client.eth_gas_price["result"].to_i(16)
    client.gas_price = ((gas_price / 1e9 * gas_premium).round()*1e9).to_i
    contract.gas_price = ((gas_price / 1e9 * gas_premium).round()*1e9).to_i

    # create key from private_key
    bot_key = Eth::Key.new priv: bot_prviate_key
    bot_address = bot_key.address
    contract.key = bot_key

    current_epoch = contract.call.current_epoch


    total_time = 0
    total_count = 0
    $count =0
    repeat_time_write.times do |i| 0
        time = write_test(client,read_client,contract,align_time)
        total_count=total_count+1
        total_time=total_time+time
    end
    $count = $count - repeat_time_write

    puts "#{url} - align_time #{align_time} - write avg  #{ (total_time.to_f / total_count).round(2)} latency"
    puts "------------------------------------------"
end


["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"]
pass time since last block mined 3.209117438 save_last_block_numbe 15237771  last_block_number 15237771  compare true
pass time since last block mined 0.310357419 save_last_block_numbe 15237772  last_block_number 15237772  compare true
[align time shift] last_block_number 15237772 - time 0.500017992 
latency 2 = tx_block_number 15237774 - last_block_number 15237772
pass time since last block mined 0.725430807 save_last_block_numbe 15237774  last_block_number 15237774  compare true
pass time since last block mined 0.288378632 save_last_block_numbe 15237775  last_block_number 15237775  compare true
[align time shift] last_block_number 15237775 - time 0.500039403 
latency 2 = tx_block_number 15237777 - last_block_number 15237775
pass time since last block mined 0.870427839 save_last_block_numbe 15237777  last_block_number 15237777  compare true
pass time since last block mined 0.305391776 save_last_blo

[["chainstack-us", "https://nd-945-619-721.p2pify.com/b3ad097ddddd0e049d63d877439f4408"], ["chainstack-fullnode", "https://nd-899-804-114.p2pify.com/cd02e4da86fbd699f58552da80e8f1ec"]]

In [ ]:
# load init moudle file

load "/work/pannote/panbot_note.rb"
nil

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>